In [27]:
# This notebook converts subject 51 and 52's EEG files to bids format (subject with OS crash)

# Setup
import numpy as np
import mne
import os.path as op
from numpy.core.numeric import ones
import pandas as pd
from mne_bids import write_raw_bids, BIDSPath, print_dir_tree, read_raw_bids, mark_bad_channels
import dateutil.parser

# directories
study_name = 'Main-study'
data_dir = '/Users/yeganeh/Documents/Raw Files/PLB_HYP_OTKA/Live Sessions'
bids_root = f'/Users/yeganeh/Codes/otka-preprocessing/data/{study_name}'
docs_dir = '/Users/yeganeh/Codes/otka-preprocessing/docs'
bh_dir = '/Users/yeganeh/Codes/otka-preprocessing/behavioral_data'
fif_dir = '/Users/yeganeh/Documents/Raw Files/PLB_HYP_OTKA/fif_files'

subject_ids = ['214911', '214911relaxation', '214911embedded', '214911confusion', '214911posthypnotic',
'2133014', '2133014_relaxation', '2133014_wightnoise_posthyp', '2133014_poathynosis']

markers_code = {'baseline_1':334.0,
'intro_control':108.0, 'intro_hypnosis':103.0,
 'whitenoise':337.0, 'embedded':336.0, 'relaxation':331.0, 'confusion':363.0,
 'experience':416.0,
 'baseline_2':343.0}

Apart from looking for the eye movemnt in EEG recording, another way to have a more exact estimation of the start of inductions would be measuring the average time that it took for participants to read through the procedure introductions from 'time_procedure', ' time_Hs' and 'time_cs' variables in their behavioral data.

In [ ]:
###################################################### subject 51 #######################################################
bids_id = '51'
raw_list = list()
bids_list = list()

iterators = range(5)
runs = [0, 3, 4, 7, 8]
run_map = dict(zip(iterators, runs))

for sub_id in [subject_ids[5], subject_ids[6], subject_ids[7], subject_ids[8]]:
    fname = op.join(data_dir,f'{sub_id}.vhdr')
    raw = mne.io.read_raw_brainvision(fname, eog=('EOG1', 'EOG2'), misc= ['ECG'])
    raw.set_channel_types({'ECG':'ecg'})
    raw.info['line_freq'] = 50  # specify power line frequency

    ########################### create bids list for basline1, induction1 & experience1
    if sub_id == '2133014':
        tmin_index = raw.annotations.description.tolist().index('Stimulus/S  2')

        for iterator in iterators:

            #crop
            tmin = raw.annotations.onset[tmin_index + run_map[iterator]]
            tmax = raw.annotations.onset[tmin_index + run_map[iterator] + 1]
            if run_map[iterator] in [0,18]:
                description = f'baseline{int(iterator/9+1)}'
            elif run_map[iterator] % 2:
                description = f'induction{int((run_map[iterator] - 3)/4 + 1)}'
            else:
                description = f'experience{int(run_map[iterator]/4)}'

            raw_segment = raw.copy().crop(tmin=tmin, tmax=tmax)

            #define annotations
            onset = raw_segment.annotations.onset
            new_annot = mne.Annotations(onset=[0],
                                    duration=[onset[1]-onset[0]],
                                    description=[description])
                
            raw_segment.set_annotations(new_annot)
            
            #save 
            fname_new = op.join(fif_dir, f'{subject_ids[5]}-{description}-raw.fif')
            raw_segment.save(fname_new)
            raw_segment = mne.io.read_raw_fif(fname_new)

            #creat lists
            raw_list.append(raw_segment)
            bids_path = BIDSPath(subject=bids_id,
                                session='01',
                                task= description,
                                root=bids_root)
            bids_list.append(bids_path)

    ##################################### Add bids lists for induction3 and experience3   
    elif sub_id == '2133014_relaxation':
        # inspecting from the EEG recording the start of the confusion induction, when participants instructed to close his eyes, was 202 s:
        ########## induction3
        description = 'induction3'
        tmin_ind = 202
        tmax_ind = 202 + markers_code['relaxation']
        raw_segment = raw.copy().crop(tmin=tmin_ind, tmax=tmax_ind)

        #save 
        fname_new = op.join(fif_dir, f'{subject_ids[5]}-{description}-raw.fif')
        raw_segment.save(fname_new)
        raw_segment = mne.io.read_raw_fif(fname_new)

        #creat lists
        raw_list.append(raw_segment)
        bids_path = BIDSPath(subject=bids_id,
                            session='01',
                            task= description,
                            root=bids_root)
        bids_list.append(bids_path)

        ########## experience3
        description = 'experience3'
        tmin_exp = tmax_ind + 1
        tmax_exp = tmin_exp + markers_code['experience']
        raw_segment = raw.copy().crop(tmin=tmin_exp, tmax=tmax_exp)

        #save 
        fname_new = op.join(fif_dir, f'{subject_ids[5]}-{description}-raw.fif')
        raw_segment.save(fname_new)
        raw_segment = mne.io.read_raw_fif(fname_new)

        #creat lists
        raw_list.append(raw_segment)
        bids_path = BIDSPath(subject=bids_id,
                            session='01',
                            task= description,
                            root=bids_root)
        bids_list.append(bids_path)
    
    elif sub_id == '2133014_wightnoise_posthyp':
        ######### baseline2
        description = 'induction4'
        tmin_ind = 234
        tmax_ind = 234 + markers_code['whitenoise']
        raw_segment = raw.copy().crop(tmin=tmin_ind, tmax=tmax_ind)

        #save 
        fname_new = op.join(fif_dir, f'{subject_ids[5]}-{description}-raw.fif')
        raw_segment.save(fname_new)
        raw_segment = mne.io.read_raw_fif(fname_new)

        #creat lists
        raw_list.append(raw_segment)
        bids_path = BIDSPath(subject=bids_id,
                            session='01',
                            task= description,
                            root=bids_root)
        bids_list.append(bids_path)
    
        ########## experience3
        description = 'experience4'
        tmin_exp = tmax_ind + 1
        tmax_exp = tmin_exp + markers_code['experience']
        raw_segment = raw.copy().crop(tmin=tmin_exp, tmax=tmax_exp)

        #save 
        fname_new = op.join(fif_dir, f'{subject_ids[5]}-{description}-raw.fif')
        raw_segment.save(fname_new)
        raw_segment = mne.io.read_raw_fif(fname_new)

        #creat lists
        raw_list.append(raw_segment)
        bids_path = BIDSPath(subject=bids_id,
                            session='01',
                            task= description,
                            root=bids_root)
        bids_list.append(bids_path)
    
    else:
        ######### baseline2
        description = 'baseline2'
        tmin_base = 68
        tmax_base = 368 #from inspecting the recording
        raw_segment = raw.copy().crop(tmin=tmin_base, tmax=tmax_base)

        #save 
        fname_new = op.join(fif_dir, f'{subject_ids[5]}-{description}-raw.fif')
        raw_segment.save(fname_new)
        raw_segment = mne.io.read_raw_fif(fname_new)

        #creat lists
        raw_list.append(raw_segment)
        bids_path = BIDSPath(subject=bids_id,
                            session='01',
                            task= description,
                            root=bids_root)
        bids_list.append(bids_path)



# create bids from lists
for raw, bids_path in zip(raw_list, bids_list):
    write_raw_bids(raw, bids_path, overwrite=True)


# mark bad channels
for bids_path in bids_list:
    if 'induction3' in bids_path.basename or 'experience3' in bids_path.basename:
        mark_bad_channels(ch_names=['FC1','FC2','Fp2'], bids_path=bids_path, verbose=False, overwrite=True)
    else:
        mark_bad_channels(ch_names=['FC1','FC2'], bids_path=bids_path, verbose=False, overwrite=True)

In [ ]:
###################################################### subject 52 #######################################################
bids_id = '52'
raw_list = list()
bids_list = list()

iterators = range(3)
runs = [0, 3, 4]
run_map = dict(zip(iterators, runs))

for sub_id in [subject_ids[0], subject_ids[3], subject_ids[4]]:
    fname = op.join(data_dir,f'{sub_id}.vhdr')
    raw = mne.io.read_raw_brainvision(fname, eog=('EOG1', 'EOG2'), misc= ['ECG'])
    raw.set_channel_types({'ECG':'ecg'})
    raw.info['line_freq'] = 50  # specify power line frequency

    ########################### create bids list for basline1, induction1 & experience1
    if sub_id == '214911':
        onset = raw.annotations.onset
        desc = raw.annotations.description
        duration = raw.annotations.duration

        # estimation of the start of the first baseline from behavioral data
        # open behavioral data
        bhdata = pd.read_csv(f'{bh_dir}/subject-{subject_ids[0]}.csv')
        bh_trg1 = dateutil.parser.parse(bhdata.loc[0,'timestamp_trigger_1'])
        bh_baseline_d = dateutil.parser.parse(bhdata.loc[2,'timestamp_baseline_start']) - bh_trg1 #duration between trigger 1 and start of the baseline.
        bh_baseline_d = bh_baseline_d.total_seconds() 
        # this duration should be added to the onset of trigger 1 in the eeg trigger onsets
        index = list(desc).index('Stimulus/S  1')
        eeg_trg1 = onset[index]
        baseline_start = bh_baseline_d + eeg_trg1

        # new annotation
        new_onset = np.insert(onset, 3, baseline_start)
        new_duration = np.insert(duration, 3, 0.001)
        new_desc = np.insert(desc, 3, 'Stimulus/S  2')
        new_annot = mne.Annotations(onset = new_onset, duration = new_duration, description  = new_desc)
        raw.set_annotations(new_annot)
        tmin_index = raw.annotations.description.tolist().index('Stimulus/S  2')

        for iterator in iterators:

            #crop
            tmin = raw.annotations.onset[tmin_index + run_map[iterator]]
            tmax = raw.annotations.onset[tmin_index + run_map[iterator] + 1]
            if run_map[iterator] in [0,18]:
                description = f'baseline{int(iterator/9+1)}'
            elif run_map[iterator] % 2:
                description = f'induction{int((run_map[iterator] - 3)/4 + 1)}'
            else:
                description = f'experience{int(run_map[iterator]/4)}'

            raw_segment = raw.copy().crop(tmin=tmin, tmax=tmax)

            #define annotations
            onset = raw_segment.annotations.onset
            new_annot = mne.Annotations(onset=[0],
                                    duration=[onset[1]-onset[0]],
                                    description=[description])
                
            raw_segment.set_annotations(new_annot)
            
            #save 
            fname_new = op.join(fif_dir, f'{subject_ids[0]}-{description}-raw.fif')
            raw_segment.save(fname_new)
            raw_segment = mne.io.read_raw_fif(fname_new)

            #creat lists
            raw_list.append(raw_segment)
            bids_path = BIDSPath(subject=bids_id,
                                session='01',
                                task= description,
                                root=bids_root)
            bids_list.append(bids_path)

    ##################################### Add bids lists for induction4 and experience4 (the recordings for ind2,3 and exp2,3 are not useable)    
    elif sub_id == '214911confusion':
        # inspecting from the EEG recording the start of the confusion induction, when participants instructed to close his eyes, was 206 s:
        ########## induction4
        tmin_ind = 206
        tmax_ind = 206 + markers_code['confusion']
        description = 'induction4'
        raw_segment = raw.copy().crop(tmin=tmin_ind, tmax=tmax_ind)

        #save 
        fname_new = op.join(fif_dir, f'{subject_ids[0]}-{description}-raw.fif')
        raw_segment.save(fname_new)
        raw_segment = mne.io.read_raw_fif(fname_new)

        #creat lists
        raw_list.append(raw_segment)
        bids_path = BIDSPath(subject=bids_id,
                            session='01',
                            task= description,
                            root=bids_root)
        bids_list.append(bids_path)

        ########## experience4
        description = 'experience4'
        tmin_exp = tmax_ind + 1
        tmax_exp = tmin_exp + markers_code['experience']
        raw_segment = raw.copy().crop(tmin=tmin_exp, tmax=tmax_exp)

        #save 
        fname_new = op.join(fif_dir, f'{subject_ids[0]}-{description}-raw.fif')
        raw_segment.save(fname_new)
        raw_segment = mne.io.read_raw_fif(fname_new)

        #creat lists
        raw_list.append(raw_segment)
        bids_path = BIDSPath(subject=bids_id,
                            session='01',
                            task= description,
                            root=bids_root)
        bids_list.append(bids_path)
    
    else:
        ######### baseline2
        description = 'baseline2'
        tmin_base = 4
        tmax_base = 304 #from inspecting the recording
        raw_segment = raw.copy().crop(tmin=tmin_base, tmax=tmax_base)

        #save 
        fname_new = op.join(fif_dir, f'{subject_ids[0]}-{description}-raw.fif')
        raw_segment.save(fname_new)
        raw_segment = mne.io.read_raw_fif(fname_new)

        #creat lists
        raw_list.append(raw_segment)
        bids_path = BIDSPath(subject=bids_id,
                            session='01',
                            task= description,
                            root=bids_root)
        bids_list.append(bids_path)




# create bids from lists
for raw, bids_path in zip(raw_list, bids_list):
    write_raw_bids(raw, bids_path, overwrite=True)

# mark bad channels
mark_bad_channels(ch_names=['PO4'], bids_path=bids_path[-1], verbose=False, overwrite=True)